In [1]:
# Circulation and heat transport regimes 
import xarray as xr
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
import gsw
import cmocean

In [2]:
def along_fjord_regime(datapath, case_id, time):
    #/Users/weiyangbao/Documents/Model_outputs/
    #/work/oceans/wbao/MITgcm_results/iceplume/1_BaseCase
    file0 = xr.open_dataset(datapath + '/state_' + str(format(case_id,'03d')) + '.nc')
    # removed duplicated data caused by model restart
    file = file0.isel(T=~file0.get_index("T").duplicated())
    state = file.isel(X=range(260), Xp1=range(261), Y=range(35,45), Yp1=range(35,45)) # T=range(t0,tn)
    
    X = state.X.data 
    x_dist = X / 1000
    depth = state.Z.data
    pres = gsw.p_from_z(depth, 55)
    pt = state.Temp.data[time,:,:,:].mean(1)
    s = state.S.data[time,:,:,:].mean(1)
    p = np.tile(pres,[len(X),1]).T
    rhoA = gsw.rho(s, pt, p) - 1000
    
    # Along-fjord velocity
    u0 = state.U.data[time,:,:,:] # dims = T, Z, Y, Xp1
    u = (u0[:, :, 1:].mean(1) + u0[:, :, :-1].mean(1)) / 2
    
    # Vetical velocity
    W0 = state.W.data
    bt = np.zeros((W0.shape[0],1,W0.shape[2],W0.shape[3])) # Add vertical velocity (0 m/s) at the bottom
    W = np.concatenate((W0, bt), axis=1)
    wzy = (W[:,1:,:,:] + W[:,:-1,:,:]) / 2
    w = wzy[time,:,:,:].mean(1)
    
    #pt0 = file.Temp.data.mean(2)[1,:,:260]
    pt0 = state.Temp.data.mean(2)[0,:,:]
    mask = (pt0 == 0)
    pt[mask] = np.nan
    s[mask] = np.nan
    rhoA[mask] = np.nan
    u[mask] = np.nan
    w[mask] = np.nan
        
    return (x_dist, depth, pt, rhoA, u, w)

In [3]:
path1 = '/work/oceans/wbao/MITgcm_results/iceplume/Sal_Linear_minhs'
path2 = '/work/oceans/wbao/MITgcm_results/iceplume/Sal_Linear_maxhs'
    
x1, z1, pt1, rho1, u1, w1 = along_fjord_regime(path1, 5, -1) # 0.5N2, Regime1
x2, z2, pt2, rho2, u2, w2 = along_fjord_regime(path2, 4, 20) # 1N2, hs/hf=0.12, Regime2
x3, z3, pt3, rho3, u3, w3 = along_fjord_regime(path1, 1, 16) # 4N2, Regime3
x4, z4, pt4, rho4, u4, w4 = along_fjord_regime(path1, 1, -1) # 4N2, Regime4

In [ ]:
# Make plot for three different sill depths (New Runs)
fig1, (ax1, ax2, ax3, ax4) = plt.subplots(4,1, sharex=True, figsize=(12,16))

cm = np.linspace(8,10,20) # Contour range
cmap = cmocean.cm.thermal
#cmap = 'plasma_r'

x_dist = x1
depth = z1
im1 = ax1.contourf(x1, z1, pt1, cm, cmap=cmap) 
#ax1.axvline(x=x_dist[120], color='g',lw=2)
ax1.set_facecolor('grey')
ax1.tick_params(direction='in', labelsize=15)


cbaxes = fig1.add_axes([0.75, 0.9, 0.15, 0.01])
cbar = fig1.colorbar(im1,cax=cbaxes, orientation='horizontal')
cbar.set_ticks([8, 9, 10])
cbar.ax.set_title(r'Temperature [$^\circ C$]', size=12)


# CS1 = ax1.contour(x1,z1,rho1,5, colors='k', linewidths=.5)
# plt.clabel(CS1, fmt='%1.1f', fontsize=12, inline=1)
ax1.xaxis.set_visible(False)
ax1.set_yticks([depth[0], -100, -200, -300, depth[-1]])
ax1.set_yticklabels(["0","-100","-200","-300","-400"])
ax1.set_ylabel('Depth [$m$]', size=15)
ax1.set_title(r'(a) Regime 1', loc='left', size=15, weight='bold')


im2 = ax2.contourf(x2, z2, pt2, cm, cmap=cmap)
#ax1.axvline(x=x_dist[120], color='g',lw=2)
ax2.set_facecolor('grey')
ax2.tick_params(direction='in', labelsize=15)

# CS2 = ax2.contour(x2,z2,rho2, 5, colors='k', linewidths=.5)
# plt.clabel(CS2, fmt='%1.1f', fontsize=12, inline=1)
ax2.xaxis.set_visible(False)
ax2.set_yticks([depth[0], -100, -200, -300, depth[-1]])
ax2.set_yticklabels(["0","-100","-200","-300","-400"])
ax2.set_ylabel('Depth [$m$]', size=15)
ax2.set_title(r'(b) Regime 2', loc='left', size=15, weight='bold')




im3 = ax3.contourf(x3, z3, pt3, cm, cmap=cmap)
#ax1.axvline(x=x_dist[120], color='g',lw=2)
ax3.set_facecolor('grey')
ax3.tick_params(direction='in', labelsize=15)


# CS3 = ax3.contour(x3,z3,rho3,5, colors='k', linewidths=.5)
# plt.clabel(CS3, fmt='%1.1f', fontsize=12, inline=1)
ax3.xaxis.set_visible(False)
ax3.set_yticks([depth[0], -100, -200, -300, depth[-1]])
ax3.set_yticklabels(["0","-100","-200","-300","-400"])
ax3.set_ylabel('Depth [$m$]', size=15)
ax3.set_title(r'(c) Regime 3', loc='left', size=15, weight='bold')



im4 = ax4.contourf(x4, z4, pt4, cm, cmap=cmap)
#ax1.axvline(x=x_dist[120], color='g',lw=2)
ax4.set_facecolor('grey')
ax4.tick_params(direction='in', labelsize=15)


# CS4 = ax4.contour(x4,z4,rho4,5, colors='k', linewidths=.5)
# plt.clabel(CS4, fmt='%1.1f', fontsize=12, inline=1)
ax4.set_yticks([depth[0], -100, -200, -300, depth[-1]])
ax4.set_yticklabels(["0","-100","-200","-300","-400"])
ax4.set_ylabel('Depth [$m$]', size=15)
ax4.set_title(r'(d) Regime 4', loc='left', size=15, weight='bold')
ax4.set_xticks([x_dist[1], 5, 10, 15, x_dist[-1]])
ax4.set_xticklabels(["0","5","10","15","20"])
ax4.set_xlabel('Distance from glacier [$km$]', size=15)




sl, sr = 140, 220 # quiver over sill range

u1_sill = u1[:,sl:sr]
w1_sill = w1[:,sl:sr]
x1_sill = x1[sl:sr]

nx, nz = 10, 5

Q_1s = ax1.quiver(x1_sill[::nx],z1[::nz],u1_sill[::nz,::nx], w1_sill[::nz,::nx],scale=6, width=0.001, color='k')
u1[:,sl:sr] = 'nan'
w1[:,sl:sr] = 'nan'
Q_1 = ax1.quiver(x1[::nx],z1[::nz],u1[::nz,::nx], w1[::nz,::nx],scale=2, width=0.001, color='k')
ax1.quiverkey(Q_1, 0.7, 1.05, 0.1, r'0.1 $m\,s^{-1}$', fontproperties={'size':12}, coordinates='axes')


# u2_sill = u2[:,sl:sr]
# w2_sill = w2[:,sl:sr]
# x2_sill = x2[sl:sr]
# Q_2s = ax2.quiver(x2_sill[::nx],z2[::nz],u2_sill[::nz,::nx], w2_sill[::nz,::nx],scale=6, width=0.001, color='k')
# u2[:,sl:sr] = 'nan'
# w2[:,sl:sr] = 'nan'
u2
u2[u2<0] = u2[u2<0]*5
Q_2 = ax2.quiver(x2[::nx],z2[::nz],u2[::nz,::nx], w2[::nz,::nx],scale=2, width=0.001, color='k')


u3_sill = u3[:,sl:sr]
w3_sill = w3[:,sl:sr]
x3_sill = x3[sl:sr]
Q_3s = ax3.quiver(x3_sill[::nx],z3[::nz],u3_sill[::nz,::nx], w3_sill[::nz,::nx],scale=6, width=0.001, color='k')

u3[:,sl:sr] = 'nan'
w3[:,sl:sr] = 'nan'
u3[:,178:] = u3[:,178:] / 10
Q_3 = ax3.quiver(x3[::nx],z3[::nz],u3[::nz,::nx], w3[::nz,::nx],scale=2, width=0.001, color='k')


u4_sill = u4[:,sl:sr]
w4_sill = w4[:,sl:sr]
x4_sill = x4[sl:sr]
Q_4s = ax4.quiver(x4_sill[::nx],z4[::nz],u4_sill[::nz,::nx], w4_sill[::nz,::nx],scale=6, width=0.001, color='k')

u4[:,sl:sr] = 'nan'
w4[:,sl:sr] = 'nan'
Q_4 = ax4.quiver(x4[::nx],z4[::nz],u4[::nz,::nx], w4[::nz,::nx],scale=2, width=0.001, color='k')

plt.show()

In [ ]:
u3[:,220]

In [4]:
def planview_regime(datapath, case_id, time, depth):

    file0 = xr.open_dataset(datapath + '/state_' + str(format(case_id,'03d')) + '.nc')
    # removed duplicated data caused by model restart
    file = file0.isel(T=~file0.get_index("T").duplicated())
    state = file.isel(X=range(260), Xp1=range(261), Y=range(35,45), Yp1=range(35,46)) # T=range(t0,tn)
    
    y_dist = state.Y.data / 1000 
    x_dist = state.X.data / 1000

    pt = state.Temp.data[time,depth,:,:]
#     s = state.S.data[time,:,:,:].mean(1)
#     p = np.tile(pres,[len(X),1]).T
#     rhoA = gsw.rho(s, pt, p) - 1000
    
    # Along-fjord velocity
    u0 = state.U.data[time,depth,:,:] # dims = T, Z, Y, Xp1
    u = (u0[:, 1:] + u0[:, :-1]) / 2
    
    # Across-fjord velocity
    v0 = state.V.data[time,depth,:,:]
    v = (v0[1:,:] + v0[:-1,:]) / 2
    
        
    return (y_dist, u, v, pt)

In [ ]:
Y = xr.open_dataset(path1 + '/state_' + str(format(5,'03d')) + '.nc').Y.data
Yp = xr.open_dataset(path1 + '/state_' + str(format(5,'03d')) + '.nc').Yp1.data
#Y[35:45], Yp[35:45]
depth

In [ ]:
d1, d2, d3, d4 = 2, 17, 52, 12
y1, uz1, vz1, pz1 = planview_regime(path1, 5, -1, d1) # 0.5N2, Regime1
y2, uz2, vz2, pz2 = planview_regime(path2, 4, 20, d2) # 1N2, hs/hf=0.12, Regime2
y3, uz3, vz3, pz3 = planview_regime(path1, 1, 16, d3) # 4N2, Regime3
y4, uz4, vz4, pz4 = planview_regime(path1, 1, -1, d4) # 4N2, Regime4

In [ ]:
#uz1.shape, vz1.shape
np.linspace(15,25,6)

In [ ]:
fig2 = plt.figure(figsize=(16,8))
fig2.tight_layout()

cm = np.linspace(8,10,20) # Contour range
crho1 = [16, 18, 20, 22] # Contour range for density
crho2 = [16, 18, 20, 22, 24] # Contour range for density
cmap = cmocean.cm.thermal
x_dist = x1
depth = z1
y_dist = y1

nx, nz = 10, 5
tv_skip = (slice(None,None,10))

#### Regime 1
# Side view
ax1 = fig2.add_subplot(4,2,1)
im1 = ax1.contourf(x_dist, depth, pt1, cm, cmap=cmap) 
ax1.axhline(y=depth[d1], color='white',ls='--',lw=2)
Q1 = ax1.quiver(x_dist[::nx],depth[::nz],u1[::nz,::nx], w1[::nz,::nx],scale=5, width=0.0015)
# Isopycnals
CS1 = ax1.contour(x_dist, depth, rho1, crho1, colors='k', linewidths=.5)
plt.clabel(CS1, fmt='%1.1f', fontsize=10, inline=1)

ax1.set_facecolor('grey')
ax1.xaxis.set_visible(False)
ax1.tick_params(direction='in', labelsize=15)
ax1.set_yticks([depth[0], -200, depth[-1]])
ax1.set_yticklabels(["0","-200","-400"])
ax1.set_ylabel('Depth [$m$]', size=15)
ax1.set_title(r'(a) Regime I', loc='left', size=15, weight='bold')
ax1.set_title(r'$0.5N_0^2,h_s/h_f=0.04, Day\,60$', loc='right', size=12)
# Top view
ax2 = fig2.add_subplot(4,2,2)
im2 = ax2.contourf(x_dist, y_dist, pz1, cm, cmap=cmap) 
Q2 = ax2.quiver(x_dist[tv_skip], y_dist, uz1[:,tv_skip], vz1[:,tv_skip], scale=5, width=0.0015)
ax2.quiverkey(Q2, 0.6, 1.1, 0.2, r'0.2 $m\,s^{-1}$', fontproperties={'size':12}, coordinates='axes')
ax2.axvline(x=x_dist[178], color='grey',lw=4)
ax2.set_facecolor('grey')
ax2.xaxis.set_visible(False)
ax2.tick_params(direction='in', labelsize=15)
ax2.set_yticks([y_dist[0], (y_dist[0]+y_dist[-1])/2, y_dist[-1]])
ax2.set_yticklabels(["0","1","2"])
ax2.set_ylabel('Distance [$km$]', size=15)


#### Regime 2
# Side view
u2[u2<0] = u2[u2<0]*4
ax3 = fig2.add_subplot(4,2,3)
im3 = ax3.contourf(x_dist, depth, pt2, cm, cmap=cmap) 
ax3.axhline(y=depth[d2], color='white',ls='--',lw=2)
Q3 = ax3.quiver(x_dist[::nx],depth[::nz],u2[::nz,::nx], w2[::nz,::nx],scale=5, width=0.0015)
CS3 = ax3.contour(x_dist, depth, rho2, crho1, colors='k', linewidths=.5)
plt.clabel(CS3, fmt='%1.1f', fontsize=10, inline=1)

# Colorbar
cbaxes = fig2.add_axes([0.8, 0.93, 0.1, 0.01])
cbar = fig2.colorbar(im2,cax=cbaxes, orientation='horizontal')
cbar.set_ticks([8, 9, 10])
cbar.ax.set_title(r'Temperature [$^\circ C$]', size=12)

ax3.set_facecolor('grey')
ax3.xaxis.set_visible(False)
ax3.tick_params(direction='in', labelsize=15)
ax3.set_yticks([depth[0], -200, depth[-1]])
ax3.set_yticklabels(["0","-200","-400"])
ax3.set_ylabel('Depth [$m$]', size=15)
ax3.set_title(r'(b) Regime II', loc='left', size=15, weight='bold')
ax3.set_title(r'$1N_0^2,h_s/h_f=0.12, Day\,10$', loc='right', size=12)

# Top view
ax4 = fig2.add_subplot(4,2,4)
im4 = ax4.contourf(x_dist, y_dist, pz2, cm, cmap=cmap) 
Q4 = ax4.quiver(x_dist[tv_skip], y_dist, uz2[:,tv_skip], vz2[:,tv_skip], scale=5, width=0.0015)
ax4.axvline(x=x_dist[178], color='grey',lw=4)
ax4.set_facecolor('grey')
ax4.xaxis.set_visible(False)
ax4.tick_params(direction='in', labelsize=15)
ax4.set_yticks([y_dist[0], (y_dist[0]+y_dist[-1])/2, y_dist[-1]])
ax4.set_yticklabels(["0","1","2"])
ax4.set_ylabel('Distance [$km$]', size=15)


#### Regime 3
# Side view
ax5 = fig2.add_subplot(4,2,5)
im5 = ax5.contourf(x_dist, depth, pt3, cm, cmap=cmap) 
ax5.axhline(y=depth[d3], color='white',ls='--',lw=2)
Q5 = ax5.quiver(x_dist[::nx],depth[::nz],u3[::nz,::nx], w3[::nz,::nx],scale=5, width=0.0015)
CS5 = ax5.contour(x_dist, depth, rho3, crho2, colors='k', linewidths=.5)
plt.clabel(CS5, fmt='%1.1f', fontsize=10, inline=1)
ax5.set_facecolor('grey')
ax5.xaxis.set_visible(False)
ax5.tick_params(direction='in', labelsize=15)
ax5.set_yticks([depth[0], -200, depth[-1]])
ax5.set_yticklabels(["0","-200","-400"])
ax5.set_ylabel('Depth [$m$]', size=15)
ax5.set_title(r'(c) Regime III', loc='left', size=15, weight='bold')
ax5.set_title(r'$4N_0^2,h_s/h_f=0.04, Day\,8$', loc='right', size=12)

# Top view
ax6 = fig2.add_subplot(4,2,6)
im6 = ax6.contourf(x_dist, y_dist, pz3, cm, cmap=cmap) 
Q6 = ax6.quiver(x_dist[tv_skip], y_dist, uz3[:,tv_skip], vz3[:,tv_skip], scale=5, width=0.0015)
ax6.axvline(x=x_dist[178], color='grey',lw=4)
ax6.set_facecolor('grey')
ax6.xaxis.set_visible(False)
ax6.tick_params(direction='in', labelsize=15)
ax6.set_yticks([y_dist[0], (y_dist[0]+y_dist[-1])/2, y_dist[-1]])
ax6.set_yticklabels(["0","1","2"])
ax6.set_ylabel('Distance [$km$]', size=15)


#### Regime 4
# Side view
ax7 = fig2.add_subplot(4,2,7)
im7 = ax7.contourf(x_dist, depth, pt4, cm, cmap=cmap) 
ax7.axhline(y=depth[d4], color='white',ls='--',lw=2)
Q7 = ax7.quiver(x_dist[::nx],depth[::nz],u4[::nz,::nx], w4[::nz,::nx],scale=5, width=0.0015)
CS7 = ax7.contour(x_dist, depth, rho4, crho2, colors='k', linewidths=.5)
plt.clabel(CS7, fmt='%1.1f', fontsize=10, inline=1)
ax7.set_facecolor('grey')
ax7.tick_params(direction='in', labelsize=15)
ax7.set_yticks([depth[0], -200, depth[-1]])
ax7.set_yticklabels(["0","-200","-400"])
ax7.set_ylabel('Depth [$m$]', size=15)
ax7.set_title(r'(d) Regime IV', loc='left', size=15, weight='bold')
ax7.set_title(r'$4N_0^2,h_s/h_f=0.04, Day\,60$', loc='right', size=12)
ax7.set_xticks([x_dist[1], 5, 10, 15, x_dist[-1]])
ax7.set_xticklabels(["0","5","10","15","20"])
ax7.set_xlabel('Distance from glacier [$km$]', size=15)
# Top view
ax8 = fig2.add_subplot(4,2,8)
im8 = ax8.contourf(x_dist, y_dist, pz4, cm, cmap=cmap) 
Q8 = ax8.quiver(x_dist[tv_skip], y_dist, uz4[:,tv_skip], vz4[:,tv_skip], scale=5, width=0.0015)
ax8.axvline(x=x_dist[178], color='grey',lw=4)
ax8.set_facecolor('grey')
ax8.tick_params(direction='in', labelsize=15)
ax8.set_yticks([y_dist[0], (y_dist[0]+y_dist[-1])/2, y_dist[-1]])
ax8.set_yticklabels(["0","1","2"])
ax8.set_ylabel('Distance [$km$]', size=15)
ax8.set_xticks([x_dist[1], 5, 10, 15, x_dist[-1]])
ax8.set_xticklabels(["0","5","10","15","20"])
ax8.set_xlabel('Distance from glacier [$km$]', size=15)

# set the spacing between subplots
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.15, 
                    hspace=0.4)

In [ ]:
fig_path ="/home/1959/Jupyter_Figs/FjordModeling/"
if not os.path.exists(fig_path):
    os.makedirs(fig_path)
#fig1.savefig(fig_path + 'Regimes_Snapshots.png', dpi=300, bbox_inches='tight')
fig2.savefig(fig_path + 'Regimes_side&top.png', dpi=300, bbox_inches='tight') 